## <center>Підготовка та аналіз даних</center>
## <center>Лабораторна робота №2</center>
#### <center>ФБ-25 Кравченко Максим</center>

Імпортуємо потрібні нам бібліотеки

In [1]:
import datetime
import requests
import urllib.request
import pandas as pd
import os


Скачуємо VHI індекси для всіх областей
Також додаємо в назву дату та час скачування

In [ ]:
for ids in range(1, 28):
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={ids}&year1=1981&year2=2024&type=Mean"
    response = requests.get(url)
    if response.status_code == 200:
        if not os.path.exists('vhi'):
            os.mkdir('vhi')
            print(f'The folder is created')
        date_now = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        vhi_url = urllib.request.urlopen(url)
        file_name = f'vhi/vhi_id_{ids}_{date_now}.csv'
        out = open(file_name, 'wb')
        out.write(vhi_url.read())
        out.close()
        print(f"VHI from id {ids} was downloaded at {date_now}")
    else:
        print(f"Download complete")
        break

VHI from id 1 was downloaded at 2024-03-30_09-20-19
VHI from id 2 was downloaded at 2024-03-30_09-20-22
VHI from id 3 was downloaded at 2024-03-30_09-20-25


Даємо імена стовбцям, зчитуємо файли у фрейм

In [40]:
folder = 'vhi'
files = os.listdir(folder)
# ids = 0
df_all = pd.DataFrame()

for file_name in files:
    # ids += 1
    
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    df = pd.read_csv(f'{folder}/{file_name}', header=1, names=headers, skiprows=1)[:-1]
    df = df.drop(df.loc[df['VHI'] == -1].index)
    df['area'] = int(file_name.split("_")[2])
    # print(df.head(), '\n')
    # df_all = pd.concat([df_all, df], ignore_index=True)
    df = df.drop(columns=['empty'])
    df_all = df_all.dropna(axis=1, how='all')
    df_all = pd.concat([df_all, df]).drop_duplicates().reset_index(drop=True)
    
# zm = df_all.isna().sum()
# print(zm)
print(df_all, '\n')

       Year  Week    SMN     SMT    VCI    TCI    VHI  area
0      1982   2.0  0.063  261.53  55.89  38.20  47.04    10
1      1982   3.0  0.063  263.45  57.30  32.69  44.99    10
2      1982   4.0  0.061  265.10  53.96  28.62  41.29    10
3      1982   5.0  0.058  266.42  46.87  28.57  37.72    10
4      1982   6.0  0.056  267.47  39.55  30.27  34.91    10
...     ...   ...    ...     ...    ...    ...    ...   ...
64345  2024   8.0  0.134  281.83  62.33   3.92  33.13     9
64346  2024   9.0  0.147  283.80  61.79   4.57  33.18     9
64347  2024  10.0  0.158  285.63  59.96   6.30  33.13     9
64348  2024  11.0  0.164  286.56  56.53   9.83  33.18     9
64349  2024  12.0  0.175  287.65  55.86  13.89  34.87     9

[64350 rows x 8 columns] 


C:\Users\maxww\AppData\Local\Temp\ipykernel_22148\1112196415.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = pd.concat([df_all, df], ignore_index=True)


Змінимо індекси областей

In [13]:
dict_areas = {1: 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4, 7: 8, 8: 19, 9: 20, 10: 21, 11: 9, 13: 10, 14: 11, 15: 12,
              16: 13, 17: 15, 18: 14, 19: 16, 21: 17, 22: 18, 23: 6, 24: 1, 25: 2, 26: 7, 27: 5}
df_all["area"] = df_all["area"].replace(dict_areas)
df_all.to_csv(f'vhi/df_all.csv', index=False)

print(df_all)

       Year  Week    SMN     SMT    VCI    TCI    VHI  area
0      1982   2.0  0.063  261.53  55.89  38.20  47.04    21
1      1982   3.0  0.063  263.45  57.30  32.69  44.99    21
2      1982   4.0  0.061  265.10  53.96  28.62  41.29    21
3      1982   5.0  0.058  266.42  46.87  28.57  37.72    21
4      1982   6.0  0.056  267.47  39.55  30.27  34.91    21
...     ...   ...    ...     ...    ...    ...    ...   ...
57910  2024   8.0  0.134  281.83  62.33   3.92  33.13    20
57911  2024   9.0  0.147  283.80  61.79   4.57  33.18    20
57912  2024  10.0  0.158  285.63  59.96   6.30  33.13    20
57913  2024  11.0  0.164  286.56  56.53   9.83  33.18    20
57914  2024  12.0  0.175  287.65  55.86  13.89  34.87    20

[57915 rows x 8 columns]


Ряд VHI для області за вказаний рік, пошук екстремумів (min та max);

In [15]:
def vhi(area, year):
    return df_all[(df_all["area"] == area) & (df_all["Year"] == year)]['VHI']

def vhi_min(area, year):
    return df_all[(df_all["area"] == area) & (df_all["Year"] == year)]['VHI'].min()

def vhi_max(area, year):
    return df_all[(df_all["area"] == area) & (df_all["Year"] == year)]['VHI'].max()

df_all['Year'] = df_all['Year'].astype(int)

In [16]:
vhi(6, 2005)

31176    47.42
31177    51.20
31178    53.68
31179    54.35
31180    54.66
31181    56.10
31182    56.81
31183    56.06
31184    56.27
31185    56.51
31186    56.42
31187    56.02
31188    54.84
31189    52.12
31190    49.52
31191    48.44
31192    53.62
31193    52.32
31194    52.21
31195    51.77
31196    53.50
31197    54.49
31198    54.25
31199    55.50
31200    57.29
31201    57.93
31202    58.44
31203    65.13
31204    59.35
31205    61.40
31206    63.51
31207    64.39
31208    66.13
31209    65.04
31210    60.86
31211    57.68
31212    56.59
31213    54.98
31214    53.23
31215    52.48
31216    55.20
31217    57.22
31218    57.66
31219    58.95
31220    56.50
31221    54.77
31222    54.48
31223    53.22
31224    50.36
31225    47.41
31226    45.59
Name: VHI, dtype: float64

In [18]:
vhi_min(6, 2005)


45.59

In [8]:
vhi_max(6, 2005)

67.33

Ряд VHI за вказаний діапазон років для вказаних областей;

In [19]:
def vhi_range(year_min, year_max, areas):
    if not isinstance(areas, list) or not areas:
        return print('Empty or not a list')
    return df_all[(df_all['Year'] >= year_min) & (df_all['Year'] <= year_max) & (df_all['area'].isin(areas))][['Year', 'VHI', 'area']]

In [21]:
vhi_range(2000, 2005, [5, 2, 3])

,Year,VHI,area
35225,2000,24.65,2
35226,2000,27.49,2
35227,2000,31.36,2
35228,2000,37.28,2
35229,2000,40.85,2
...,...,...,...
48382,2005,40.70,3
48383,2005,42.92,3
48384,2005,41.13,3
48385,2005,41.49,3


Роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей

In [24]:
def extreme_droughts(procent):
    df_drouhts = df_all[(df_all['VHI'] <= 15) & (df_all['VHI'] != -1)]
    grouped = df_drouhts.groupby('Year')['area'].nunique()
    result = grouped[grouped > (25 * procent / 100)].reset_index()
    return result

In [28]:
extreme_droughts(5)

,Year,area
0,1986,2
1,1993,2
2,2000,6
3,2007,5


Аналогічно для помірних посух

In [31]:
def moderate_droughts(procent, min=15, max=40):
    df_drouhts = df_all[(df_all['VHI'] >= min) & (df_all['VHI'] <= max)]
    grouped = df_drouhts.groupby('Year')['area'].nunique()
    count_areas = df_all['area'].nunique()
    result = grouped[grouped > (count_areas * procent / 100)].reset_index()
    return result

In [32]:
moderate_droughts(20)

,Year,area
0,1982,25
1,1983,25
2,1984,25
3,1985,25
4,1986,25
5,1987,25
6,1988,25
7,1989,25
8,1990,25
9,1991,24
